In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler

In [10]:
dataset1 = pd.read_csv('data/dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('data/dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('data/dataset3.csv')

dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

dataset12 = pd.concat([dataset1,dataset2],axis=0)

In [11]:
dataset1_y = dataset1.label
dataset1_x = dataset1.drop(['User_id','label','day_gap_before','day_gap_after', 'Discount_rate', 'avg_user_date_datereceived_gap'],axis=1)  # 'day_gap_before','day_gap_after' cause overfitting, 0.77
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['User_id','label','day_gap_before','day_gap_after', 'Discount_rate', 'avg_user_date_datereceived_gap'],axis=1)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['User_id','label','day_gap_before','day_gap_after', 'Discount_rate', 'avg_user_date_datereceived_gap'],axis=1)
dataset3_preds = dataset3[['User_id','Coupon_id','Date_received']]
dataset3_x = dataset3.drop(['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after', 'Discount_rate', 'user_date_datereceived_gap'],axis=1)

print (dataset1_x.shape,dataset2_x.shape,dataset3_x.shape)

(134416, 51) (252481, 51) (112803, 51)


In [12]:
dataset3_x.head()

,Distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,discount_rate,total_sales,sales_use_coupon,total_coupon,...,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,1.0,12,12,30.0,5.0,1,0.833333,7757.0,1239.0,35933.0,...,1,1,0,0,1,0,0,0,0,0
1,NaN,6,6,30.0,5.0,1,0.833333,119.0,17.0,100.0,...,1,1,0,0,0,1,0,0,0,0
2,5.0,27,27,200.0,20.0,1,0.900000,2280.0,100.0,3529.0,...,2,1,0,0,0,1,0,0,0,0
3,5.0,27,27,100.0,10.0,1,0.900000,2280.0,100.0,3529.0,...,2,1,0,0,0,1,0,0,0,0
4,2.0,8,8,30.0,1.0,1,0.966667,1065.0,182.0,795.0,...,1,1,0,0,0,0,0,1,0,0


In [13]:
dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)

In [14]:
params={'booster':'gbtree',
    'objective': 'rank:pairwise',
    'eval_metric':'auc',
    'gamma':0.1,
    'min_child_weight':1.1,
    'max_depth':5,
    'lambda':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'eta': 0.01,
    'tree_method':'exact',
    'seed':0,
    'nthread':12
    }

In [15]:
model = xgb.train(params,dataset12,num_boost_round=1000)

[10:49:17] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-auc:0.834279
[10:49:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-auc:0.837939
[10:49:19] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-auc:0.84453
[10:49:19] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-auc:0.844603
[10:49:20] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-auc:0.847136
[10:49:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-auc:0.84776
[10:49:21] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree prunin

[10:49:55] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[53]	train-auc:0.855989
[10:49:56] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[54]	train-auc:0.855956
[10:49:57] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[55]	train-auc:0.855961
[10:49:57] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[56]	train-auc:0.856158
[10:49:58] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[57]	train-auc:0.85614
[10:49:59] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[58]	train-auc:0.856159
[10:49:59] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree

[10:50:34] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[106]	train-auc:0.857949
[10:50:34] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[107]	train-auc:0.857992
[10:50:35] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[108]	train-auc:0.858004
[10:50:36] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[109]	train-auc:0.858043
[10:50:37] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[110]	train-auc:0.858074
[10:50:38] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[111]	train-auc:0.858215
[10:50:39] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

[10:51:10] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[159]	train-auc:0.859632
[10:51:11] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[160]	train-auc:0.859653
[10:51:12] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[161]	train-auc:0.859675
[10:51:12] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[162]	train-auc:0.8597
[10:51:13] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[163]	train-auc:0.859712
[10:51:14] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[164]	train-auc:0.859724
[10:51:15] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74:

[10:51:49] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[212]	train-auc:0.861075
[10:51:50] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[213]	train-auc:0.861128
[10:51:51] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[214]	train-auc:0.861138
[10:51:52] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[215]	train-auc:0.861175
[10:51:52] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[216]	train-auc:0.861207
[10:51:53] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[217]	train-auc:0.861297
[10:51:54] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

...

[10:58:04] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[742]	train-auc:0.876711
[10:58:04] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[743]	train-auc:0.876726
[10:58:05] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[744]	train-auc:0.876744
[10:58:06] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[745]	train-auc:0.876755
[10:58:06] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[746]	train-auc:0.876765
[10:58:07] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[747]	train-auc:0.876778
[10:58:08] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

[10:58:39] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[795]	train-auc:0.87768
[10:58:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[796]	train-auc:0.877695
[10:58:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[797]	train-auc:0.877731
[10:58:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[798]	train-auc:0.87775
[10:58:42] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[799]	train-auc:0.877768
[10:58:43] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[800]	train-auc:0.877781
[10:58:43] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74:

[10:59:15] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[848]	train-auc:0.878566
[10:59:15] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[849]	train-auc:0.878581
[10:59:16] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[850]	train-auc:0.878598
[10:59:17] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[851]	train-auc:0.878607
[10:59:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[852]	train-auc:0.878618
[10:59:18] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[853]	train-auc:0.878633
[10:59:19] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

[11:00:03] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[901]	train-auc:0.879372
[11:00:04] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[902]	train-auc:0.879382
[11:00:05] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[903]	train-auc:0.879392
[11:00:05] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[904]	train-auc:0.879408
[11:00:06] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[905]	train-auc:0.879422
[11:00:07] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[906]	train-auc:0.879431
[11:00:08] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

[11:00:40] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[954]	train-auc:0.880091
[11:00:41] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[955]	train-auc:0.880097
[11:00:42] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[956]	train-auc:0.880105
[11:00:42] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[957]	train-auc:0.880109
[11:00:43] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[958]	train-auc:0.880125
[11:00:44] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[959]	train-auc:0.880131
[11:00:45] C:\dev\libs\xgboost\src\tree\updater_prune.cc:7

In [40]:
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label.reshape(-1, 1))
dataset3_preds.sort_values(by=['Coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds_0.8806.csv",index=None,header=None)
dataset3_preds.describe()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


,User_id,Coupon_id,Date_received,label
count,1.128030e+05,112803.000000,1.128030e+05,112803.000000
mean,3.684618e+06,9064.658006,2.016072e+07,0.319309
std,2.126358e+06,4147.283515,9.017693e+00,0.160940
min,2.090000e+02,3.000000,2.016070e+07,0.000000
25%,1.843824e+06,5035.000000,2.016071e+07,0.208431
50%,3.683073e+06,9983.000000,2.016072e+07,0.288529
75%,5.525176e+06,13602.000000,2.016072e+07,0.401352
max,7.361024e+06,14045.000000,2.016073e+07,1.000000


In [41]:
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)

In [22]:
sorted(dataset3_preds.label, reverse=True)

[1.0,
 0.97465849,
 0.96996486,
 0.9686693,
 0.96824193,
 0.96744359,
 0.96686471,
 0.96680713,
 0.96667266,
 0.96643806,
 0.96355081,
 0.96282077,
 0.96019733,
 0.95930994,
 0.95843554,
 0.95826733,
 0.95644569,
 0.95611811,
 0.95576847,
 0.95548928,
 0.95545232,
 0.95512414,
 0.95490289,
 0.95482373,
 0.9546622,
 0.95351577,
 0.95198882,
 0.94807923,
 0.94787896,
 0.94765651,
 0.94739878,
 0.94720173,
 0.94691253,
 0.94253349,
 0.94140112,
 0.94096518,
 0.94078791,
 0.93986011,
 0.93919086,
 0.93904543,
 0.93877685,
 0.93723917,
 0.93659055,
 0.93597174,
 0.9339087,
 0.93360889,
 0.93134809,
 0.92951286,
 0.92928874,
 0.92806506,
 0.92573571,
 0.92570794,
 0.92565429,
 0.92540026,
 0.92464995,
 0.92440045,
 0.92440045,
 0.92395353,
 0.92376125,
 0.92370331,
 0.92368484,
 0.92365026,
 0.92355442,
 0.92354035,
 0.9235003,
 0.9228462,
 0.9228009,
 0.92254913,
 0.92205834,
 0.92184544,
 0.92170262,
 0.92162883,
 0.92151964,
 0.9204123,
 0.91999364,
 0.91981065,
 0.91904616,
 0.91893899,
